# Optimal well disposition for the Darcy problem

First we import some of the standard modules.

In [ ]:
import numpy as np
import porepy as pp
import scipy.sparse as sps
import scipy

import sys

Depending on the setting, we then need to setup the local path for importing some useful functions

In [ ]:
main_folder = "./"
spe10_folder = main_folder + "spe10"
sys.path.insert(1, spe10_folder)

from functions import *
from spe10 import Spe10

In [ ]:
def solve_fine(spe10, pos_well, injection_rate=1, well_pressure=0, export_folder=None):
    """
    Compute the averaged gradient and flux for a given subdomain and direction of the pressure
    gradient.

    Args:
        spe10 (object): The object representing the subdomain.
        pos_well (np.ndarray): The position of the production well.
        injection_rate (float, optional): The injection rate of the wells. Defaults to 1.
        well_pressure (float, optional): The pressure at the production well. Defaults to 0.
        export_folder (str, optional): If given, path where to export the results. Defaults to
            None.

    Returns:
        float: The maximum pressure at the injection wells.
    """
    # Extract the grid for simplicity
    sd = spe10.sd
    perm_dict = spe10.perm_as_dict()

    # Permeability
    perm_tensor = pp.SecondOrderTensor(kxx=perm_dict["kxx"])
    print(perm_tensor)

    # Boundary conditions
    b_faces = sd.tags["domain_boundary_faces"].nonzero()[0]

    # Define the labels and values for the boundary faces
    labels = np.array(["neu"] * b_faces.size)
    bc_val = np.zeros(sd.num_faces)
    bc = pp.BoundaryCondition(sd, b_faces, labels)

    # Collect all parameters in a dictionary
    key = "flow"
    parameters = {"second_order_tensor": perm_tensor, "bc": bc, "bc_values": bc_val}
    data = pp.initialize_default_data(sd, {}, key, parameters)

    # Discretize the problem
    discr = pp.Mpfa(key)
    discr.discretize(sd, data)

    A, b = discr.assemble_matrix_rhs(sd, data)

    # Add the injection wells, all with the same injection rate
    b_wells = np.zeros_like(b)
    index_iwells = [
        0,
        spe10.full_shape[0] - 1,
        spe10.full_shape[0] * spe10.full_shape[1] - spe10.full_shape[0],
        spe10.full_shape[0] * spe10.full_shape[1] - 1,
    ]
    b_wells[index_iwells] = injection_rate

    # Add the production well by using a Lagrange multiplier, first we identify the cell
    ij_well = np.floor((np.asarray(pos_well) / spe10.spacing[:-1])).astype(int)
    print(ij_well)
    index_pwell = spe10.full_shape[0] * ij_well[1] + ij_well[0]
    vect = np.zeros((sd.num_cells, 1))
    vect[index_pwell] = 1

    # Solve the linear system and compute the pressure by adding the constraint
    A = sps.bmat([[A, vect], [vect.T, None]], format="csc")
    b = np.append(b + b_wells, well_pressure)
    p = sps.linalg.spsolve(A, b)[:-1]

    # extract the discretization matrices build 
    mat_discr = data[pp.DISCRETIZATION_MATRICES][key]

    # reconstruct the flux as post-process
    q_tpfa = mat_discr["flux"] @ p + mat_discr["bound_flux"] @ bc_val

    # to export the flux                                                                                                   
    mvem = pp.MVEM(key)                                                                                      
    mvem.discretize(sd, data)                                                                                                                                                                                     
    # construct the P0 flux reconstruction                                                                                 
    cell_q_mpfa = mvem.project_flux(sd, q_tpfa, data) 
    

    # Export the solution
    if export_folder is not None:
        save = pp.Exporter(sd, "sol", folder_name=export_folder)
        save.write_vtu([("p", p), ("log_kxx", np.log10(perm_dict["kxx"])),("q_mpfa", cell_q_mpfa)])

    # Return the maximum pressure at the injection wells
    return np.max(p[index_iwells])

In [ ]:
def upscale(sd, perm, dir, export_folder=None):
    """
    Compute the averaged gradient and flux for a given subdomain and direction of the pressure
    gradient.

    Args:
        sd (pp.Grid): The grid representing the subdomain.
        perm (dict): The permeability of the subdomain divided in the fields "kxx" and "kyy"
        dir (int): The direction of the flow, 0 means x-direction and 1 means y-direction.
        export_folder (str): If given, path where to export the results.
            Default to None, no exporting.

    Returns:
        (np.ndarray, np.ndarray): averaged gradient and flux.
    """
    # TODO: from checkpoint 1

In [ ]:
def compute_tensor(grad_h, grad_v, q_h, q_v):
    """
    Compute the upscaled permeability tensor.

    Args:
        grad_h (np.ndarray): Gradient in the horizontal direction.
        grad_v (np.ndarray): Gradient in the vertical direction.
        q_h (np.ndarray): Flux in the horizontal direction.
        q_v (np.ndarray): Flux in the vertical direction.

    Returns:
        perm (np.ndarray): Upscaled permeability tensor.

    The function solves a linear system to obtain the upscaled permeability tensor
    based on the given gradients and fluxes. It enforces numerical symmetry and
    checks if the resulting tensor is symmetric positive definite (SPD).
    """
    # Solve the linear system to get the upscaled permeability
    
    # TODO: from checkpoint 1

In [ ]:
selected_layers = 35
folder_results = main_folder + "results/"

# Read the SPE10 grid
spe10 = Spe10(selected_layers)

# Read the permeability associated to the given layer(s)
perm_folder = spe10_folder + "/perm/"
spe10.read_perm(perm_folder)
perm_dict = spe10.perm_as_dict()

# Partition the grid
num_part = 10 * 10
part, sub_sds, sd_coarse = coarse_grid(spe10.sd, num_part)

# Define the upscaled permeability
kxx_up = np.zeros(spe10.sd.num_cells)
kxy_up = np.zeros(spe10.sd.num_cells)
kyx_up = np.zeros(spe10.sd.num_cells)
kyy_up = np.zeros(spe10.sd.num_cells)

kxx = np.zeros(spe10.sd.num_cells)

result = []
# Loop over the subdomains
for sub_sd_id, sub_sd in enumerate(sub_sds):
    # Extract the permeability values associated to the current subdomain
    mask = part == sub_sd_id
    sub_perm = {key: val[mask] for key, val in perm_dict.items()}
    kxx[mask] = sub_perm["kxx"]

    # Upscale the permeability in the x-direction
    folder = folder_results + str(sub_sd_id) + "_x"
    q_h, grad_h = upscale(sub_sd, sub_perm, 0, folder)

    # Upscale the permeability in the y-direction
    folder = folder_results + str(sub_sd_id) + "_y"
    q_v, grad_v = upscale(sub_sd, sub_perm, 1, folder)

    # Compute the upscaling permeability tensor
    kk = compute_tensor(grad_h, grad_v, q_h, q_v)

    # Save the data
    kxx_up[mask] = kk[0]
    kxy_up[mask] = kk[1]
    kyx_up[mask] = kk[2]
    kyy_up[mask] = kk[3]

    result.append([kk[0], kk[1], kk[2], kk[3]])

    print(f"Subdomain {sub_sd_id}: {kk}")

result = np.array(result)

Define the function that given a subdomain, its data, and a direction compute the upscaled gradient and flux.

In [ ]:
def solve_coarse(
    sd,
    pos_well,
    kk,
    injection_rate=1,
    well_pressure=0,
    export_folder=None,
):
    """
    Compute the averaged gradient and flux for a given subdomain and direction of the pressure
    gradient.

    Args:
        spe10 (object): The object representing the subdomain.
        pos_well (np.ndarray): The position of the production well.
        injection_rate (float, optional): The injection rate of the wells. Defaults to 1.
        well_pressure (float, optional): The pressure at the production well. Defaults to 0.
        export_folder (str, optional): If given, path where to export the results. Defaults to
            None.

    Returns:
        float: The maximum pressure at the injection wells.
    """
    # TODO: adapt solve_fine with upscaled quantity

Perform the evaluation of the functional for the Spe10 benchmark of a given layer.

In [ ]:
selected_layers = 35
folder_results = main_folder + "results/"

# Read the SPE10 grid
spe10 = Spe10(selected_layers)

# Read the permeability associated to the given layer(s)
perm_folder = spe10_folder + "/perm/"
spe10.read_perm(perm_folder)
# Define the function to evaluate that depends only on the position of the injection well
CostFunctional = lambda x: solve_coarse(
    sd_coarse, x, result, export_folder=folder_results
)

CostFunctionalFine = lambda x: solve_fine(
    spe10, x, export_folder=folder_results
)

print(CostFunctional([231, 481]))
print(CostFunctionalFine([231, 481]))


In [ ]:
def Checkpoint1_solution(selected_layers, folder_results):
    # Read the SPE10 grid
    spe10 = Spe10(selected_layers)

    # Read the permeability associated to the given layer(s)
    perm_folder = spe10_folder + "/perm/"
    spe10.read_perm(perm_folder)
    perm_dict = spe10.perm_as_dict()

    # Partition the grid
    num_part = 20
    part, sub_sds, sd_coarse = coarse_grid(spe10.sd, num_part)

    # Define the upscaled permeability
    kxx_up = np.zeros(spe10.sd.num_cells)
    kxy_up = np.zeros(spe10.sd.num_cells)
    kyx_up = np.zeros(spe10.sd.num_cells)
    kyy_up = np.zeros(spe10.sd.num_cells)

    kxx = np.zeros(spe10.sd.num_cells)

    result = []
    # Loop over the subdomains
    for sub_sd_id, sub_sd in enumerate(sub_sds):
        # Extract the permeability values associated to the current subdomain
        mask = part == sub_sd_id
        sub_perm = {key: val[mask] for key, val in perm_dict.items()}
        kxx[mask] = sub_perm["kxx"]

        # Upscale the permeability in the x-direction
        folder = folder_results + str(sub_sd_id) + "_x"
        q_h, grad_h = upscale(sub_sd, sub_perm, 0, folder)

        # Upscale the permeability in the y-direction
        folder = folder_results + str(sub_sd_id) + "_y"
        q_v, grad_v = upscale(sub_sd, sub_perm, 1, folder)

        # Compute the upscaling permeability tensor
        kk = compute_tensor(grad_h, grad_v, q_h, q_v)

        # Save the data
        kxx_up[mask] = kk[0]
        kxy_up[mask] = kk[1]
        kyx_up[mask] = kk[2]
        kyy_up[mask] = kk[3]

        result.append([kk[0], kk[1], kk[2], kk[3]])

        print(f"Subdomain {sub_sd_id}: {kk}")

    return sd_coarse, result

# Optimization

In [ ]:


def checkpoint2_solution(selected_layers, folder_results):

     # Read the SPE10 grid
    spe10 = Spe10(selected_layers)

    # compute upscaling
    sd_coarse, result = Checkpoint1_solution(selected_layers, folder_results)

    # define cost functional on coarse scale (note that it depends on the upscaling)
    CostFunctional = lambda x: solve_coarse(sd_coarse, x, result, export_folder=folder_results)

    # define cost functional on fine scale 
    CostFunctionalFine = lambda x: solve_fine(spe10, x, export_folder=folder_results)

    # stating point
    mu_guess = [
        (spe10.full_physdims[0]) / 2.0,
        (spe10.full_physdims[1]) / 2.0,
    ]

    # TODO using both solve_corse and solve_fine
    CostFunctional (mu_guess)
    mu_est = mu_guess

    return mu_est


